In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
import gensim

print(np.__version__)
print(pd.__version__)
print(tf.__version__)
print(nltk.__version__)
print(gensim.__version__)

1.21.4
1.3.3
2.6.0
3.6.5
3.8.1


In [3]:
!pip install gensim==3.8.1

### Step1. 데이터 다운로드

In [4]:
from requests import get
import os


os.getcwd()

'/aiffel/aiffel/workplace/Aiffel_Quest/GoingDeeper/GD06'

In [5]:
# os.mkdir('./data')
def download(url, file_name):
    with open(file_name, "wb") as file:   
        response = get(url)
        file.write(response.content)
        
url = 'https://raw.githubusercontent.com/songys/Chatbot_data/master'
file_name = 'ChatbotData.csv'

download(f'{url}/{file_name}', f'./data/{file_name}')

In [6]:
os.listdir('./data')

['ChatbotData.csv']

In [8]:
df = pd.read_csv(f'./data/ChatbotData.csv')

df.tail()

,Q,A,label
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2
11822,힘들어서 결혼할까봐,도피성 결혼은 하지 않길 바라요.,2


In [12]:
len(df)

TypeError: object of type 'NoneType' has no len()

In [10]:
df.drop_duplicates(['Q'], inplace=True)
df.drop_duplicates(['A'], inplace=True)

len(df)

7731

### Step 2. 데이터 정제

In [11]:
import re

def preprocess_sentence(sentence):
    
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!가-힣ㄱ-ㅎㅏ-ㅣ0-9]+", " ", sentence)
    sentence = sentence.strip()
    
    return sentence

In [12]:
df['Q'] = df['Q'].apply(lambda it: preprocess_sentence(it))
df['A'] = df['A'].apply(lambda it: preprocess_sentence(it))

df.head()

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
4,ppl 심하네,눈살이 찌푸려지죠.
5,sd카드 망가졌어,다시 새로 사는 게 마음 편해요.


In [13]:
questions = df['Q']
answers = df['A']

len(questions), len(answers)

(7731, 7731)

### Step 3. 데이터 토큰화

In [15]:
from konlpy.tag import Mecab
import numpy as np


def build_corpus(src_data, tgt_data):
    
    mecab = Mecab()
    
    def get_morphs(s):
        return mecab.morphs(s)
    
    mecab_src_corpus = list(map(get_morphs, src_data))
    mecab_tgt_corpus = list(map(get_morphs, src_data))
    
    mecab_num_tokens = [len(s) for s in mecab_src_corpus] + [len(s) for s in mecab_tgt_corpus]
    
    # 최대 길이를 (평균 + 2*표준편차)로 계산
    max_len = round(np.mean(mecab_num_tokens) + 2 * np.std(mecab_num_tokens))
    print(f'max_len : {max_len}')
    
    src_corpus, tgt_corpus = [], []
    for q, a in zip(mecab_src_corpus, mecab_tgt_corpus):
        if len(q) <= max_len and len(a) <= max_len:
            if q not in src_corpus and a not in tgt_corpus:
                src_corpus.append(q)
                tgt_corpus.append(a)
    
    return src_corpus, tgt_corpus

que_corpus, ans_corpus = build_corpus(questions, answers)

max_len : 15


In [16]:
len(que_corpus), len(ans_corpus)

(7429, 7429)

### Step4. Augmentation

Lexical Substitution을 실제로 적용해보자. <br>


In [18]:
from gensim.models.word2vec import Word2Vec

w2v_model = Word2Vec.load('/aiffel/aiffel/transformer_chatbot/data/word2vec_ko.model')

In [19]:
import random

def lexical_sub(sentence, word2vec):
    try:
        _from = random.choice(sentence)
        _to = word2vec.wv.most_similar(_from)[0][0]
    except:
        return None
    
    res = []
    for x in sentence:
        if x is _from: res.append(_to)
        else: res.append(x)

    return res

In [25]:
from tqdm import tqdm_notebook


def augment_corpus(src_corpus, tgt_corpus, wv):
    new_src_corpus = []
    new_tar_corpus = []
    corpus_size = len(src_corpus)
    
    for i in tqdm_notebook(range(corpus_size)):
        q = src_corpus[i]
        a = tgt_corpus[i]
        
        new_src = lexical_sub(q, wv)
        new_tar = lexical_sub(a, wv)
        
        if new_src: 
            new_src_corpus.append(new_src)
            new_tar_corpus.append(a)
            
        if new_tar: 
            new_src_corpus.append(q)
            new_tar_corpus.append(new_tar)
    
    return new_src_corpus, new_tar_corpus

In [26]:
aug_q, aug_a = augment_corpus(que_corpus, ans_corpus, w2v_model)

que_corpus += aug_q
ans_corpus += aug_a
    
len(que_corpus), len(ans_corpus)

/tmp/ipykernel_31/1664475964.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(corpus_size)):


  0%|          | 0/7429 [00:00<?, ?it/s]

(22057, 22057)

### Step5. 데이터 벡터화

ans_corpus에 <start>토큰과 <end> 토큰이 추가되지 않은 상태이니 이를 먼저 해결한 후 벡터화를 진행한다.

In [28]:
ans_corpus = [["<start>"] + ans + ["<end>"] for ans in ans_corpus]

In [29]:
import tensorflow as tf  


data = que_corpus + ans_corpus

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=None, filters=' ', oov_token='<unk>')
tokenizer.fit_on_texts(data)

enc_train = tokenizer.texts_to_sequences(que_corpus)
enc_train = tf.keras.preprocessing.sequence.pad_sequences(enc_train, padding='post')

dec_train = tokenizer.texts_to_sequences(ans_corpus)
dec_train = tf.keras.preprocessing.sequence.pad_sequences(dec_train, padding='post')

In [30]:
enc_train.shape, dec_train.shape

((22057, 15), (22057, 17))

In [31]:
vocab_size = len(tokenizer.index_word) + 2

vocab_size

5903

### Step 6. 훈련하기

In [32]:
def positional_encoding(pos, d_model):
    
    def cal_angle(position, i):
        return position / np.power(10000, int(i)/d_model)  # np.power(a,b) > a^b(제곱)
    
    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]
    
    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])
    
    return sinusoid_table

In [33]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)  
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        
        QK = tf.matmul(Q, K, transpose_b=True)
        scaled_qk = QK / tf.math.sqrt(d_k)
        
        if mask is not None:
            scaled_qk += (mask * -1e9)
        
        attentions = tf.nn.softmax(scaled_qk, axis=-1)

        out = tf.matmul(attentions, V)
        return out, attentions
    
    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])
        return split_x
    
    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))
        return combined_x
    
    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask
        )
        
        out = self.combine_heads(out)
        out = self.linear(out)
        
        return out, attention_weights

In [34]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.w_1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.w_2 = tf.keras.layers.Dense(d_model)
        
    def call(self, x):
        out = self.w_1(x)
        out = self.w_2(out)
        return out

In [35]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)
        
        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):

        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn

In [36]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, enc_out, causality_mask, padding_mask):

        residual = x
        out = self.norm_1(x)

        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual
        
        residual = out
        out = self.norm_2(out)

        out, dec_enc_attn = self.enc_dec_attn(out, enc_out, enc_out, causality_mask)
        out = self.do(out)
        out += residual

        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

In [37]:
class Encoder(tf.keras.Model):
    def __init__(self, n_layers, d_model, n_heads, d_ff, dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)]
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        out = x
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
            
        return out, enc_attns

In [38]:
class Decoder(tf.keras.Model):
    def __init__(self, n_layers, d_model, n_heads, d_ff, dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)]
        
    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = self.dec_layers[i](out, enc_out, causality_mask, padding_mask)
            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)
        
        return out, dec_attns, dec_enc_attns

In [39]:
class Transformer(tf.keras.Model):
    def __init__(self, n_layers, d_model, n_heads, d_ff, src_vocab_size, tgt_vocab_size,
                 pos_len, dropout=0.2, shared=True):
        super(Transformer, self).__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        
        self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
        self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)
        
        self.pos_encoding = positional_encoding(pos_len, d_model)
        
        self.do = tf.keras.layers.Dropout(dropout)
        
        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)
        
        self.fc = tf.keras.layers.Dense(tgt_vocab_size)
        
        self.shared = shared
        
        if shared:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))
        
        
    def embedding(self, emb, x):
        
        seq_len = x.shape[1]
        out = emb(x)
        
        if self.shared:
            out *= tf.math.sqrt(self.d_model)
        
        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)
        
        return out
    
    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):

        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)

        dec_out, dec_attns, dec_enc_attns = self.decoder(dec_in, enc_out, causality_mask, dec_mask)

        logits = self.fc(dec_out)
        return logits, enc_attns, dec_attns, dec_enc_attns

In [40]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(src_len, tgt_len):
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)

    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [41]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [47]:
# 주어진 하이퍼파라미터로 Transformer 인스턴스 생성


VOCAB_SIZE = 20000

transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.3,
    # shared_fc=True,
    # shared_emb=True
    shared=True)

d_model = 512

print("슝=3")

슝=3


In [48]:
# Learning Rate 인스턴스 선언 & Optimizer 구현
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)


print("슝=3")

슝=3


In [49]:
# Loss Function 정의
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)
print("슝=3")

슝=3


In [50]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns
print("슝=3")

슝=3


In [52]:
EPOCHS = 5 
for epoch in range(EPOCHS):
    total_loss = 0
    tqdm_bar = tqdm(total=len(train_dataset))

    for (batch, (src, tgt)) in enumerate(train_dataset):
        tgt_in = tgt[:, :-1]  # Decoder의 input
        gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

        enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

        with tf.GradientTape() as tape:
            predictions, _, _, _ = transformer(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
            loss = loss_function(gold, predictions)

        gradients = tape.gradient(loss, transformer.trainable_variables)
        optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

        total_loss += loss
        tqdm_bar.update(1)
        tqdm_bar.set_description(f'Epoch {epoch + 1}, Loss: {total_loss / (batch + 1):.4f}')

    tqdm_bar.close()

    # 검증 데이터셋을 사용하여 에포크 종료 후 모델 성능 평가 가능

    # 에포크 종료 후 손실 출력
    print(f'Epoch {epoch + 1}, Loss: {total_loss / len(train_dataset):.4f}')

NameError: name 'train_dataset' is not defined

In [42]:
from tqdm.notebook import tqdm
import random
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

class ChatBot():
    def __init__(self, tokenizer, enc_data, dec_data,
                 n_layers=6, d_model=512, n_heads=8, d_ff=2048,
                 vocab_size=20000, pos_len=200, dropout=0.3, shared=True,
                 epochs=20, batch_size=64):
        super(ChatBot, self).__init__()
        
        self.tokenizer = tokenizer
        self.enc_data = enc_data
        self.dec_data = dec_data
        
        self.n_layers = n_layers
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_ff = d_ff
        self.dropout = dropout
        
        self.model = Transformer(
            n_layers=n_layers,
            d_model=d_model,
            n_heads=n_heads,
            d_ff=d_ff,
            src_vocab_size=vocab_size,
            tgt_vocab_size=vocab_size,
            pos_len=pos_len,
            dropout=dropout,
            shared=shared
        )
        
        self.EPOCHS = epochs
        self.BATCH_SIZE = batch_size
        
        learning_rate = LearningRateScheduler(d_model)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
        self.loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
        
        self.examples = [
            "말하기 전에 생각했나요? 입에서 때가 나오네요.",
            "논문은 쓰셨나요? ",
            "무슨 영화를 제일 좋아하세요? 부귀영화요.",
            "굳이? 구지? 구찌."
        ]
        
    def loss_function(self, real, pred):
        mask = tf.math.logical_not(tf.math.equal(real, 0))
        loss_ = self.loss_object(real, pred)
        
        mask = tf.cast(mask, dtype=loss_.dtype)
        loss_ *= mask

        return tf.reduce_sum(loss_)/tf.reduce_sum(mask)
    
    
    @tf.function()
    def train_step(self, src, tgt):
        gold = tgt[:, 1:]

        enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt)

        with tf.GradientTape() as tape:
            predictions, enc_attns, dec_attns, dec_enc_attns = self.model(src, tgt, enc_mask, dec_enc_mask, dec_mask)
            loss = self.loss_function(gold, predictions[:, :-1])

        gradients = tape.gradient(loss, self.model.trainable_variables)    
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))

        return loss, enc_attns, dec_attns, dec_enc_attns
        
        
    def train(self):
        
        for epoch in range(self.EPOCHS):
            total_loss = 0

            idx_list = list(range(0, self.enc_data.shape[0], self.BATCH_SIZE))   
            random.shuffle(idx_list)
            t = tqdm(idx_list)
            
            for (batch, idx) in enumerate(t):
                batch_loss, enc_attns, dec_attns, dec_enc_attns = self.train_step(
                    self.enc_data[idx:idx+self.BATCH_SIZE],
                    self.dec_data[idx:idx+self.BATCH_SIZE])
                
                total_loss += batch_loss

                t.set_description_str('Epoch %2d' % (epoch + 1))
                t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))
                
                
    def evaluate(self, sentence):
        mecab = Mecab()
        sentence = mecab.morphs(preprocess_sentence(sentence))
        sentence = self.tokenizer.texts_to_sequences(sentence)
        _input = tf.keras.preprocessing.sequence.pad_sequences(
            sentence, maxlen=self.enc_data.shape[-1], padding='post')

        ids = []
        output = tf.expand_dims([self.tokenizer.word_index['<start>']], 0)
        for i in range(self.dec_data.shape[-1]):
            enc_padding_mask, combined_mask, dec_padding_mask = generate_masks(_input, output)

            predictions, enc_attns, dec_attns, dec_enc_attns = self.model(
                _input, output, enc_padding_mask, combined_mask, dec_padding_mask)
            
            predicted_id = tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()
            
            if tokenizer.word_index['<end>'] == predicted_id:
                result = ' '.join(self.tokenizer.sequences_to_texts([ids]))
                return result, enc_attns, dec_attns, dec_enc_attns
            
            ids.append(predicted_id)
            output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)
    
        result = ' '.join(self.tokenizer.sequences_to_texts([ids]))
        return result, enc_attns, dec_attns, dec_enc_attns
        
        
    def chat(self, sentence):
        result, enc_attns, dec_attns, dec_enc_attns = self.evaluate(sentence)
        
        print(f'Q: {sentence}')
        print(f'A: {result}')
       
        
    def eval_bleu_single(self, src_sentence, tgt_sentence):
        src_tokens = self.tokenizer.texts_to_sequences(src_sentence)
        tgt_tokens = self.tokenizer.texts_to_sequences(tgt_sentence)

        if (len(src_tokens) > self.enc_data.shape[-1]): return None
        if (len(tgt_tokens) > self.enc_data.shape[-1]): return None

        reference = tgt_sentence.split()
        candidate, _, _, _ = self.evaluate(src_sentence)
        
        score = sentence_bleu([reference], candidate,
                              smoothing_function=SmoothingFunction().method1)

        return score


    def eval_bleu(self, src_sentences, tgt_sentence):
        total_score = 0.0
        sample_size = len(src_sentences)

        for idx in tqdm(range(sample_size)):
            score = self.eval_bleu_single(src_sentences[idx], tgt_sentence[idx])
            if not score: continue

            total_score += score
            
        return total_score / sample_size
        
    
    def submit(self, src_sentences, tgt_sentence):
        print('# 제출')
        print()
        
        print('Translations')
        for i, s in enumerate(self.examples):
            print(f'> {i + 1}')
            self.chat(s)
            print()
            
        print('BLEU Score')
        print(f'score: {self.eval_bleu(src_sentences, tgt_sentence)}')
        print()

        print('Hyperparameters')
        print(f'> n_layers: {self.n_layers}')
        print(f'> d_model: {self.d_model}')
        print(f'> n_heads: {self.n_heads}')
        print(f'> d_ff: {self.d_ff}')
        print(f'> dropout: {self.dropout}')
        print()
        
        print('Training Parameters')
        print(f'> Batch Size: {self.BATCH_SIZE}')
        print(f'> Epoch At: {self.EPOCHS}')

### Step 7. 성능 측정하기

In [43]:
model_1 = ChatBot(tokenizer, enc_train, dec_train)
model_1.train()

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

...

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

In [53]:
model_1.submit(list(questions[:100]), list(answers[:100]))

# 제출

Translations
> 1
Q: 지루하다, 놀러가고 싶어.
A: 고백

> 2
Q: 오늘 일찍 일어났더니 피곤하다.
A: 오늘 오늘 오늘 오늘 오늘 오늘 오늘 오늘 오늘

> 3
Q: 간만에 여자친구랑 데이트 하기로 했어.
A: 간만에

> 4
Q: 집에 있는다는 소리야.
A: 집 집 집

BLEU Score


  0%|          | 0/100 [00:00<?, ?it/s]

score: 0.0

Hyperparameters
> n_layers: 6
> d_model: 512
> n_heads: 8
> d_ff: 2048
> dropout: 0.3

Training Parameters
> Batch Size: 64
> Epoch At: 20


In [54]:
model_2 = ChatBot(tokenizer, enc_train, dec_train, n_layers=6, n_heads=16, d_model=1024, d_ff=4096)
model_2.train()

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

...

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/345 [00:00<?, ?it/s]

In [55]:
model_2.submit(list(questions[:100]), list(answers[:100]))

# 제출

Translations
> 1
Q: 지루하다, 놀러가고 싶어.
A: 지루 지루 낮잠 지루

> 2
Q: 오늘 일찍 일어났더니 피곤하다.
A: 오늘 피곤

> 3
Q: 간만에 여자친구랑 데이트 하기로 했어.
A: 간만에 간만에 간만에

> 4
Q: 집에 있는다는 소리야.
A: 집 힘내

BLEU Score


  0%|          | 0/100 [00:00<?, ?it/s]

score: 0.0

Hyperparameters
> n_layers: 6
> d_model: 1024
> n_heads: 16
> d_ff: 4096
> dropout: 0.3

Training Parameters
> Batch Size: 64
> Epoch At: 20
